In [ ]:
import torch
import torch.nn as nn
from torchvision import transforms
from PIL import Image
from CNN_architecture import PneumoniaCNN

# Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def load_model(model_path):
    """Load trained model from checkpoint"""
    checkpoint = torch.load(model_path, map_location=device)
    
    # Get hyperparameters
    hyperparams = checkpoint['hyperparameters']
    
    # Create model
    model = PneumoniaCNN(
        num_filters=hyperparams['num_filters'],
        dropout_rate=hyperparams['dropout_rate']
    ).to(device)
    
    # Load weights
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()
    
    print(f"Model loaded successfully!")
    print(f"Test Accuracy: {checkpoint['test_accuracy']:.2f}%")
    print(f"Test F1 Score: {checkpoint['test_f1']:.4f}")
    
    return model


def predict(model, image_path):
    """Predict pneumonia from chest X-ray image"""
    
    # Image preprocessing
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    # Load and preprocess image
    image = Image.open(image_path).convert('RGB')
    image_tensor = transform(image).unsqueeze(0).to(device)
    
    # Predict
    with torch.no_grad():
        outputs = model(image_tensor)
        probabilities = torch.softmax(outputs, dim=1)
        predicted_class = torch.argmax(probabilities, dim=1).item()
        confidence = probabilities[0][predicted_class].item() * 100
    
    # Results
    result = "PNEUMONIA" if predicted_class == 1 else "NORMAL"
    
    print(f"\nPrediction: {result}")
    print(f"Confidence: {confidence:.2f}%")
    print(f"Probabilities: Normal={probabilities[0][0].item()*100:.2f}%, Pneumonia={probabilities[0][1].item()*100:.2f}%")
    
    return result, confidence


if __name__ == '__main__':
    # Paths
    MODEL_PATH = r'pneumonia_cnn_best_20251028_152313.pth'
    IMAGE_PATH = r'chest_xray\train\PNEUMONIA\person1_bacteria_2.jpeg'  # Update with your image
    
    # Load model
    model = load_model(MODEL_PATH)
    
    # Predict
    result, confidence = predict(model, IMAGE_PATH)

Model loaded successfully!
Test Accuracy: 78.21%
Test F1 Score: 0.8479

Prediction: PNEUMONIA
Confidence: 93.86%
Probabilities: Normal=6.14%, Pneumonia=93.86%


In [ ]:
model_state = model.state_dict()

In [ ]:
checkpoint = torch.load('pneumonia_cnn_best_20251028_152313.pth')

# Now extract hyperparameters
best_hyperparams = checkpoint['hyperparameters']


{'learning_rate': 0.0005902552855603126, 'batch_size': 16, 'num_filters': 16, 'dropout_rate': 0.20231447421237492, 'weight_decay': 0.0005252316752006056, 'augmentation': {'horizontal_flip': True, 'rotation': 0, 'brightness': 0, 'contrast': 0}}


Model loaded successfully!
Test Accuracy: 94.65%
Test F1 Score: 0.9100

Prediction: PNEUMONIA
Confidence: 93.86%
Probabilities: Normal=6.14%, Pneumonia=93.86%
